In [1]:
import pandas as pd
import numpy as np
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import pickle
BATCH_SIZE=16
# root_path=r"H:\work\rm\dataset\lesspU\Dataset\Tissue Specific\Human_Liver"
root_path="/home/wls/test/DNABERT/multirm/hm6A"
import pandas as pd

# 读取正样本
with open(root_path+"/positive.txt", "r") as f:
    positive_list = f.read().splitlines() # 按行读取，并去除每行末尾的换行符

# 读取负样本
with open(root_path+"/negative.txt", "r") as f:
    negative_list = f.read().splitlines()

# 合并成一个列表
samples = [("P-" + str(i), s) for i, s in enumerate(positive_list)] + [("N-" + str(i), s) for i, s in enumerate(negative_list)]

# 转换成DataFrame
df = pd.DataFrame(samples, columns=["name", "seq"])
df["label"] = df["name"].apply(lambda x: 1 if x.startswith("P") else 0) # 添加标签列
df = df[["name", "seq", "label"]] # 调整列的顺序

# 查看DataFrame
print(df.head())

  name                                                seq  label
0  P-0  AGTTCTACATAATTACACCAAGGAGACATACAAAAACACACAGCGC...      1
1  P-1  AGCTCTTAATTACTAGGCCACACTGACTGTTAAGCAGTTAGCAGTG...      1
2  P-2  CTCCGGTGATTTTTAAACCTGAAAAACTGAAATGCTTTGGAAATAT...      1
3  P-3  CTGGAATGGAAAGTTTTTAAGGCAAACCTTTATCGGAAAATGCATA...      1
4  P-4  TACTCAAAGTTTAGTTGAAATTTTAACTAGTTTTTGGTTTGTTTAA...      1


In [2]:
def seq2kmer(seq):
    k=3
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers


In [3]:
df["sequence"]=df["seq"].apply(seq2kmer)


In [5]:
import random
data=df[["sequence","label"]]
data=data.sample(frac=1)
data.reset_index(drop=True,inplace=True)
training_ratio = 0.7;testing_ratio = 1-training_ratio
random_bools = [random.random() < training_ratio for _ in range(len(data))]
distribution = ["training" if b else "testing" for b in random_bools]
data["train"]=distribution
data.loc[data.train=="training"][["sequence","label"]].to_csv("/home/wls/test/DNABERT/examples/sample_data/hm6a/train.tsv",sep="\t",index=None)


In [6]:
data.loc[data.train=="testing"][["sequence","label"]].to_csv("/home/wls/test/DNABERT/examples/sample_data/hm6a/dev.tsv",sep="\t",index=None)


In [ ]:
cd examples

export KMER=3
export MODEL_PATH=/home/wls/test/DNABERT/3-new-12w-0
export DATA_PATH=/home/wls/test/DNABERT/examples/sample_data/hm6a
export OUTPUT_PATH=./ft/hm6a

python run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna3 \
    --model_name_or_path $MODEL_PATH \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir $DATA_PATH \
    --max_seq_length 100 \
    --per_gpu_eval_batch_size=32   \
    --per_gpu_train_batch_size=32   \
    --learning_rate 2e-4 \
    --num_train_epochs 5.0 \
    --output_dir $OUTPUT_PATH \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8